In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

X_train_kaggle = train.drop(labels=["label"], axis=1)
X_test_kaggle = test
y_train_kaggle = train["label"]

In [4]:
train.shape, test.shape

((42000, 785), (28000, 784))

In [5]:
X_train_kaggle.shape

(42000, 784)

In [6]:
X_train_kaggle = X_train_kaggle.to_numpy().reshape(-1, 28, 28)

In [7]:
X_train_kaggle.shape

(42000, 28, 28)

In [8]:
num_classes = len(set(y_train_kaggle))
num_classes

10

From TensorFlow dataset.

In [9]:
(X_tr, y_tr), (X_tt, y_tt) = mnist.load_data()

X_train_tf = np.concatenate([X_tr, X_tt], axis=0)
y_train_tf = np.concatenate([y_tr, y_tt], axis=0)

11490434/11490434 [==============================] - 0s 0us/step


In [10]:
X_train_tf.shape, y_train_tf.shape

((70000, 28, 28), (70000,))

# Preparing the datasets

In [11]:
X_train = np.concatenate([X_train_kaggle, X_train_tf], axis = 0)
y_train = np.concatenate([y_train_kaggle, y_train_tf], axis = 0)

In [12]:
X_train = X_train.reshape(-1, 28, 28, 1)
test = test.to_numpy().reshape(-1, 28, 28, 1)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42)

In [14]:
X_train.shape, X_val.shape

((89600, 28, 28, 1), (22400, 28, 28, 1))

In [15]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes = num_classes)
y_val = to_categorical(y_val, num_classes = num_classes)

In [16]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((89600, 28, 28, 1), (22400, 28, 28, 1), (89600, 10), (22400, 10))

In [17]:
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.10,
        width_shift_range=0.2,
        height_shift_range=0.2)

In [18]:
train_gen = datagen.flow(X_train, y_train, batch_size = 128)
val_gen = datagen.flow(X_val, y_val, batch_size = 128)

In [19]:
train_gen, val_gen

(<keras.preprocessing.image.NumpyArrayIterator at 0x7fdde8832800>,
 <keras.preprocessing.image.NumpyArrayIterator at 0x7fde8adba9b0>)

In [20]:
inputs = layers.Input(shape = (28, 28, 1), dtype = tf.float32)
x = layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu', input_shape=(28, 28, 1))(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(pool_size = 2)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(pool_size = 2)(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation = 'softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(loss = 'categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(),
             metrics = ['accuracy',
                       tf.keras.metrics.Precision(name = 'precision'),
                       tf.keras.metrics.Recall(name = 'recall')])

EarlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                            patience = 10,
                                            verbose = 0,
                                            restore_best_weights = True)

In [21]:
history = model.fit(train_gen,
                    epochs=10,
                    validation_data= val_gen,
                    verbose=1,
                    callbacks=[EarlyStop])

Epoch 1/10


2023-07-16 17:56:52.010301: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


700/700 [==============================] - 53s 54ms/step - loss: 1.0055 - accuracy: 0.7198 - precision: 0.7629 - recall: 0.6888 - val_loss: 0.2914 - val_accuracy: 0.9100 - val_precision: 0.9282 - val_recall: 0.8931
Epoch 2/10
700/700 [==============================] - 38s 54ms/step - loss: 0.3793 - accuracy: 0.8827 - precision: 0.9003 - recall: 0.8689 - val_loss: 0.1971 - val_accuracy: 0.9387 - val_precision: 0.9484 - val_recall: 0.9305
Epoch 3/10
700/700 [==============================] - 38s 54ms/step - loss: 0.2747 - accuracy: 0.9156 - precision: 0.9271 - recall: 0.9067 - val_loss: 0.1363 - val_accuracy: 0.9598 - val_precision: 0.9670 - val_recall: 0.9542
Epoch 4/10
700/700 [==============================] - 38s 55ms/step - loss: 0.2355 - accuracy: 0.9282 - precision: 0.9372 - recall: 0.9215 - val_loss: 0.1172 - val_accuracy: 0.9642 - val_precision: 0.9690 - val_recall: 0.9594
Epoch 5/10
700/700 [==============================] - 39s 55ms/step - loss: 0.2123 - accuracy: 0.9359 - pre

In [22]:
y_pred_probs = model.predict(test)

875/875 [==============================] - 2s 2ms/step


In [23]:
submission = model.predict(test)
submission = np.argmax(submission, axis=1)
submission = pd.Series(submission, name="Label")
submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), submission], axis=1)
submission

875/875 [==============================] - 2s 2ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [24]:
submission.to_csv('submission.csv', index = False)